In [ ]:
import pandas as pd

from statistics import geometric_mean
from scipy.stats import percentileofscore

In [ ]:
import json
with open('../outputs/regular_season_2023_2024.json', 'r') as regular_season_file:
    regular_season = json.load(regular_season_file)

In [ ]:
with open('../outputs/save_weather_2023_2024.json', 'r') as regular_season_weather_file:
    regular_season_weather = json.load(regular_season_weather_file)

In [ ]:
regular_season_weather = [json.loads(x) for x in regular_season_weather]
low_temps = [x.get('weather_summary').get('temp_low_f') for x in regular_season_weather]
precips = [x.get('weather_summary').get('precipitation_mm') for x in regular_season_weather]

score = [geometric_mean([percentileofscore(precips, x.get('weather_summary').get('precipitation_mm')), (101-percentileofscore(low_temps, x.get('weather_summary').get('temp_low_f')))]) for x in regular_season_weather]
percentile_score = [percentileofscore(score, x) for x in score]

In [ ]:
def parse_regular_season(game: dict) -> dict:
    return {
        "gameId": game.get("gameId"),
        "awayTeam": game.get("awayTeam").get("teamTricode"),
    }

def parse_weather(game: dict) -> dict:
    return {
        "gameId": game.get("game_id"),
        "gameCode": game.get("game_code"),
        "homeTeam": game.get("home_team").get("teamTricode"),
        "venue_name": game.get("venue").get("name"),
        "homeDateTime": game.get("home_datetime")[0:10],
        "high_temperature": game.get("weather_summary").get("temp_high_f"),
        "low_temperature": game.get("weather_summary").get("temp_low_f"),
        "precipitation (mm)": game.get("weather_summary").get("precipitation_mm"),
    }

In [ ]:
rsw_df = pd.DataFrame([parse_weather(x) for x in regular_season_weather])
rs_df = pd.DataFrame([parse_regular_season(x) for x in regular_season])

In [ ]:
merged = pd.merge(rsw_df, rs_df)
merged = merged[['awayTeam', 'homeTeam', 'venue_name', 'homeDateTime', 'high_temperature', 'low_temperature', 'precipitation (mm)']]
merged['percentile_score'] = percentile_score
merged['score'] = score

In [ ]:
merged.to_csv('../outputs/game_data_2023_2024.csv', index=False)